<a href="https://colab.research.google.com/github/Ihalagedara/optimization_agent/blob/main/anomaly_detection_autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Mobitel anomaly detection/compilied_data.csv')

In [4]:
df.head(5)

,interrathigh,idlehigh,bhpriority,poli1,poli2,poli3,poli4,poli5,poli6,hoallowed0,...,cellhoparacfg12.3,cellhoparacfg12.4,cellhoparacfg12.5,cellhoparacfg12.6,cellhoparacfg12.7,cellhoparacfg12.8,cellhoparacfg12.9,cellhoparacfg12.10,cellhoparacfg12.11,cellhoparacfg12.12
0,1,1,1,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.shape

(613, 35)

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# -----------------------------
# 1. Load Data
# -----------------------------
# Good KPI parameter sets (rows = cells, cols = params)
df_good = df[:500]

# Degraded parameter set (one row)
df_bad = df[500:]

# -----------------------------
# 2. Preprocessing
# -----------------------------
scaler = StandardScaler()
X_good = scaler.fit_transform(df_good.values)
X_bad = scaler.transform(df_bad.values)

n_features = X_good.shape[1]

In [8]:
input_layer = Input(shape=(n_features,))
encoded = Dense(64, activation="relu")(input_layer)
encoded = Dense(32, activation="relu")(encoded)
bottleneck = Dense(8, activation="relu")(encoded)  # compressed representation
decoded = Dense(32, activation="relu")(bottleneck)
decoded = Dense(64, activation="relu")(decoded)
output_layer = Dense(n_features, activation="linear")(decoded)

autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse")

In [13]:
history = autoencoder.fit(
    X_good, X_good,
    epochs=100,
    batch_size=16,
    shuffle=True,
    validation_split=0.1,
    verbose=1
)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [14]:
reconstructed = autoencoder.predict(X_bad)
reconstruction_error = np.abs(X_bad - reconstructed)

4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 159ms/step


In [17]:
reconstruction_error

array([[3.03829907e-04, 2.49897243e-04, 1.06655061e-05, ...,
        3.68228182e-04, 1.14466995e-04, 1.60947442e-04],
       [2.61323186e-03, 2.00878115e-03, 4.75663692e-04, ...,
        1.74166635e-04, 2.55327672e-04, 1.57713890e-04],
       [2.61323186e-03, 2.00878115e-03, 4.75663692e-04, ...,
        1.74166635e-04, 2.55327672e-04, 1.57713890e-04],
       ...,
       [2.61323186e-03, 2.00878115e-03, 4.75663692e-04, ...,
        1.74166635e-04, 2.55327672e-04, 1.57713890e-04],
       [2.61323186e-03, 2.00878115e-03, 4.75663692e-04, ...,
        1.74166635e-04, 2.55327672e-04, 1.57713890e-04],
       [7.13941309e-02, 8.26087448e-02, 1.00597736e+00, ...,
        1.17958877e-02, 4.05496731e-03, 1.05226450e-02]])

In [24]:
type(X_bad)

numpy.ndarray

In [38]:
#changed
b = [1,	1,	1,	1,	0,	0,	1,	0,	1,	1,	0,	0,	1,	0,	0,	0,	1,	0,	0,	0,	1,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0]

In [53]:
b = [0,	1,	1,	1,	0,	0,	1,	0,	1,	1,	0,	0,	1,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0]


In [ ]:
interrathigh	idlehigh	bhpriority	poli1	poli2	poli3	poli4	poli5	poli6	hoallowed0	hoallowed1	hoallowed2	hoallowed3	hoallowed4	hoallowed5	hoallowed6	hoallowed7	hoallowed8	hoallowed9	hoallowed10	hoallowed11	hoallowed12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12	cellhoparacfg12


In [54]:
d = np.array([b])

In [63]:
dd = pd.DataFrame(d)

In [55]:
reconstructed = autoencoder.predict(d)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [56]:
reconstructed

array([[ 0.01057847,  0.11509593,  0.04610909, -0.00757766, -0.06593242,
        -0.00915979,  0.04999941,  0.04839317,  0.01867294,  0.01196197,
        -0.01513952,  0.4057234 , -0.06261214,  0.2163817 ,  0.58893675,
         0.00311109,  0.36671767,  0.33289212, -0.04696115,  0.02141835,
        -0.00910146,  0.1271523 ,  0.0456312 , -0.05507861,  0.04968323,
         0.03455656,  0.03300887, -0.04164477, -0.05103331,  0.00432462,
         0.0196552 ,  0.02406366,  0.06514403,  0.04418479,  0.06074928]],
      dtype=float32)

In [57]:
reconstruction_error = np.abs(d - reconstructed)

In [58]:
reconstruction_error

array([[0.01057847, 0.88490407, 0.95389091, 1.00757766, 0.06593242,
        0.00915979, 0.95000059, 0.04839317, 0.98132706, 0.98803803,
        0.01513952, 0.40572339, 1.06261214, 0.2163817 , 0.58893675,
        0.00311109, 0.36671767, 0.33289212, 0.04696115, 0.02141835,
        0.00910146, 0.12715229, 0.0456312 , 0.05507861, 0.04968323,
        0.03455656, 0.03300887, 0.04164477, 0.05103331, 0.00432462,
        0.0196552 , 0.02406366, 0.06514403, 0.04418479, 0.06074928]])

In [44]:
param_errors = reconstruction_error[0]
param_importance = np.argsort(param_errors)[::-1]

In [62]:
d[0]

array([0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [64]:
param_errors = reconstruction_error[0]
param_importance = np.argsort(param_errors)[::-1]

print("\nParameters ranked by abnormality (highest error first):")
for idx in param_importance:
    col = df_good.columns[idx]
    diff = d[0][idx] - reconstructed[0][idx]
    direction = "Increase" if diff < 0 else "Decrease"
    print(f"- {col}: {direction} (Current={dd.iloc[0, idx]}, Suggested≈{scaler.inverse_transform(reconstructed)[0][idx]:.1f})")


Parameters ranked by abnormality (highest error first):
- hoallowed3: Decrease (Current=1, Suggested≈0.9)
- poli1: Decrease (Current=1, Suggested≈1.0)
- hoallowed0: Decrease (Current=1, Suggested≈1.0)
- poli6: Decrease (Current=1, Suggested≈1.0)
- bhpriority: Decrease (Current=1, Suggested≈1.0)
- poli4: Decrease (Current=1, Suggested≈1.0)
- idlehigh: Decrease (Current=1, Suggested≈0.4)
- hoallowed5: Increase (Current=0, Suggested≈0.2)
- hoallowed2: Increase (Current=0, Suggested≈0.8)
- hoallowed7: Increase (Current=0, Suggested≈0.8)
- hoallowed8: Increase (Current=0, Suggested≈0.8)
- hoallowed4: Increase (Current=0, Suggested≈0.7)
- hoallowed12: Increase (Current=0, Suggested≈0.0)
- poli2: Decrease (Current=0, Suggested≈-0.1)
- cellhoparacfg12.10: Increase (Current=0, Suggested≈0.1)
- cellhoparacfg12.12: Increase (Current=0, Suggested≈0.1)
- cellhoparacfg12.1: Decrease (Current=0, Suggested≈-0.1)
- cellhoparacfg12.6: Decrease (Current=0, Suggested≈-0.1)
- cellhoparacfg12.2: Increase (

In [16]:
suggested_params = scaler.inverse_transform(reconstructed)
suggested_df = pd.DataFrame(suggested_params, columns=df_good.columns)

print("\nSuggested new parameter values:")
print(suggested_df.round(2))


Suggested new parameter values:
     interrathigh  idlehigh  bhpriority  poli1  poli2  poli3  poli4  poli5  \
0            1.00      1.00        1.00   1.00    0.0  -0.00   1.00    1.0   
1           -0.00     -0.00        1.00   1.00   -0.0  -0.00   1.00    1.0   
2           -0.00     -0.00        1.00   1.00   -0.0  -0.00   1.00    1.0   
3           -0.00     -0.00        1.00   1.00   -0.0  -0.00   1.00    1.0   
4           -0.00     -0.00        1.00   1.00   -0.0  -0.00   1.00    1.0   
..            ...       ...         ...    ...    ...    ...    ...    ...   
108          1.01      0.99        1.01   0.98    0.0  -0.01   1.00    1.0   
109         -0.00     -0.00        1.00   1.00   -0.0  -0.00   1.00    1.0   
110         -0.00     -0.00        1.00   1.00   -0.0  -0.00   1.00    1.0   
111         -0.00     -0.00        1.00   1.00   -0.0  -0.00   1.00    1.0   
112         -0.03     -0.04        1.01   1.01   -0.0  -0.02   1.01    1.0   

     poli6  hoallowed0  ...  c